In [1]:
# basic imports
import os
import numpy as np
import pandas as pd
import random
import itertools
from tqdm.notebook import tqdm
import math

# metrics calculation
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import KFold, StratifiedKFold

# basic plotting library
import matplotlib.pyplot as plt

In [2]:
class CFG:
    # pipeline parameters
    SEED        = 42
    NUM_CLASSES = 5
    TGT_LABEL   = 'label'
    N_FOLDS     = 5 
    VAL_BATCH_SIZE  = 32
    SIZE             = [512,512]
    NUM_WORKERS      = 4
   
    # tf_efficientnet_b3_ns, vit_base_patch16_384, resnext50_32x4d
    MODEL_ARCH  = 'tf_efficientnet_b4_ns'           
    
    # eff_b3_baseline, vit_baseline, resnext50_baseline_v2, , resnext50_32x4d_baseline
    WGT_MODEL   = 'eff_b4_baseline'  
    

TRAIN_PATH = '../input/cassava-leaf-disease-classification/train_images'
NPY_FOLDER = '../input/cassava-npy-train-images/train_npy_images'
DIR_INPUT  = '../input/cassava-leaf-disease-classification'

index_label_map = {
                0: "Cassava Bacterial Blight (CBB)", 
                1: "Cassava Brown Streak Disease (CBSD)",
                2: "Cassava Green Mottle (CGM)", 
                3: "Cassava Mosaic Disease (CMD)", 
                4: "Healthy"
                }

class_names = [value for key,value in index_label_map.items()]

In [3]:
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [4]:
def get_oof_stats(oof, model_name):
        
    # foldwise accuracy
    #unique_folds = oof.fold.unique()
    #for fold in unique_folds:
    #    fold_labels = oof[oof['fold']==fold].label.values
    #    fold_preds =  np.argmax(oof[oof['fold']==fold].iloc[:, 4:].values, axis=1)
    #    print(f"Fold {fold} acccuracy = {accuracy_score(fold_labels, fold_preds)}")
    
    # Overall accuracy
    predicted_label = np.argmax(oof.iloc[:, 4:].values, axis=1)
    accuracy = accuracy_score(oof['label'].values, predicted_label)
    print(f'Accuracy score for {model_name} OOF = {accuracy * 100.0}')

    # confusion matrix
    cm = confusion_matrix(predicted_label, oof['label'].values)
    class_wise_acc = []
    for i, val in enumerate(cm):
        class_wise_acc.append(val[i]/sum(val)*100)
    print(f"Classwise_acc for {model_name} = {class_wise_acc}")
    
    #plt.figure(figsize=(8,8))
    #plot_confusion_matrix(cm, class_names, normalize=True)
    #return accuracy, cm

In [5]:
oof_dataset_path = 'OOF'
model_list = ['resnext50_32x4d_baseline', 'resnext50_baseline_v2', 'vit_baseline', 
              'eff_b3_baseline', 'eff_b4_baseline'] 

oof_files = [f'{oof_dataset_path}/{model}_OOF.csv' for model in model_list]
print(f'There are {len(oof_files)} OOF files in input dataset')
oof_csv = {}
for idx,file in enumerate(oof_files):
    oof_csv[model_list[idx]] = pd.read_csv(file)

for key in oof_csv.keys():
    print(f'{key} OOF shape = {oof_csv[key].shape}')

There are 5 OOF files in input dataset
resnext50_32x4d_baseline OOF shape = (21397, 9)
resnext50_baseline_v2 OOF shape = (21397, 9)
vit_baseline OOF shape = (21397, 9)
eff_b3_baseline OOF shape = (21397, 9)
eff_b4_baseline OOF shape = (21397, 9)


In [6]:
oof_csv['eff_b3_baseline'].head()

,Unnamed: 0,fold,files,label,preds0,preds1,preds2,preds3,preds4
0,12839,3,1000015157.npy,0,0.280500,0.148493,0.265875,0.023807,0.281325
1,8560,2,1000201771.npy,3,0.000111,0.003378,0.002023,0.993812,0.000676
2,8561,2,100042118.npy,1,0.005702,0.320358,0.029464,0.026395,0.618081
3,4280,1,1000723321.npy,1,0.001122,0.986286,0.002689,0.003097,0.006805
4,8562,2,1000812911.npy,3,0.000123,0.000193,0.000119,0.998498,0.001066


In [7]:
csv0 = oof_csv['resnext50_32x4d_baseline']
csv1 = oof_csv['resnext50_baseline_v2']
csv2 = oof_csv['vit_baseline']
csv3 = oof_csv['eff_b3_baseline']
csv4 = oof_csv['eff_b4_baseline']

In [8]:
#print(np.array_equal(csv0.files.values, csv1.files.values))
#print(np.array_equal(csv0.files.values, csv2.files.values))
#print(np.array_equal(csv0.files.values, csv3.files.values))
#print(np.array_equal(csv0.files.values, csv4.files.values))

get_oof_stats(csv0, 'resnext50_32x4d_baseline')
#[0.886682, 0.899065, 0.892732, 0.888525, 0.892966], 0.891994

get_oof_stats(csv1, 'resnext50_baseline_v2')
# [0.893692, 0.901168, 0.896237, 0.888759, 0.8960037], 0.89517194

get_oof_stats(csv2, 'vit_baseline')
#[0.8929906, 0.894626, 0.887357, 0.883384, 0.887591], 0.8891897199

get_oof_stats(csv3, 'eff_b3_baseline')
# [0.893224, 0.8981308, 0.8927319, 0.886655, 0.887123], 0.89157294

get_oof_stats(csv4, 'eff_b4_baseline')
# [0.8887850, 0.8957943, 0.8913297, 0.8836176, 0.8887590], 0.88965712

Accuracy score for resnext50_32x4d_baseline OOF = 94.98060475767632
Classwise_acc for resnext50_32x4d_baseline = [84.8512173128945, 93.34264432029795, 91.14561477129669, 98.22961781019679, 87.71186440677965]
Accuracy score for resnext50_baseline_v2 OOF = 91.62499415805954
Classwise_acc for resnext50_baseline_v2 = [69.99182338511855, 90.10880316518298, 87.90820829655782, 96.26175197731682, 81.88405797101449]
Accuracy score for vit_baseline OOF = 93.76080758984905
Classwise_acc for vit_baseline = [84.375, 91.4577530176416, 90.79646017699115, 96.45669291338582, 87.78904665314403]
Accuracy score for eff_b3_baseline OOF = 90.94265551245502
Classwise_acc for eff_b3_baseline = [76.53806047966631, 86.80037313432835, 84.08231835363293, 96.09632067005684, 79.18142463597009]
Accuracy score for eff_b4_baseline OOF = 90.98939103612655
Classwise_acc for eff_b4_baseline = [69.45642795513373, 85.4954954954955, 87.71377137713772, 96.37757402675484, 80.04016064257029]


In [9]:
OOF_CSV = [csv0, csv1, csv2, csv3, csv4]
OOF = oof_files

In [10]:
x = np.zeros(( len(OOF), len(OOF_CSV[0]), 5))
print(x.shape)

(5, 21397, 5)


In [11]:
for k in range(len(OOF)):
    x[k, :, :] = OOF_CSV[k].iloc[:, 4:].values
TRUE = OOF_CSV[0].label.values

In [12]:
TRUE

array([0, 3, 1, ..., 1, 4, 4])

In [13]:
all = []
for k in range(x.shape[0]):
    acc_score = accuracy_score(TRUE, np.argmax(x[k], axis=1))
    all.append(acc_score)
    print('Model %i has OOF acc_score = %.4f'%(k,acc_score))
    
m = [np.argmax(all)]; w = []

Model 0 has OOF acc_score = 0.9498
Model 1 has OOF acc_score = 0.9162
Model 2 has OOF acc_score = 0.9376
Model 3 has OOF acc_score = 0.9094
Model 4 has OOF acc_score = 0.9099


In [14]:
m

[0]

In [15]:
old = np.max(all); 

RES = 200; 
PATIENCE = 100; 
TOL = 0.0001
DUPLICATES = False

print('Ensemble acc_score = %.4f by beginning with model %i'%(old,m[0]))
print()

for kk in range(len(OOF)):
    
    # BUILD CURRENT ENSEMBLE
    md = x[m[0],:,:]
    print(md.shape)
    for i,k in enumerate(m[1:]):
        md = w[i]*x[k,:,:] + (1-w[i])*md
        
    # FIND MODEL TO ADD
    mx = 0; mx_k = 0; mx_w = 0
    print('Searching for best model to add... ')
    
    # TRY ADDING EACH MODEL
    for k in range(x.shape[0]):
        print(k,', ',end='')
        if not DUPLICATES and (k in m): continue
            
        # EVALUATE ADDING MODEL K WITH WEIGHTS W
        bst_j = 0; bst = 0; ct = 0
        for j in range(RES):
            tmp = j/RES*x[k,:,:] + (1-j/RES)*md
            acc = accuracy_score(TRUE,np.argmax(tmp, axis=1))
            if acc>bst:
                bst = acc
                bst_j = j/RES
            else: ct += 1
            if ct>PATIENCE: break
        if bst>mx:
            mx = bst
            mx_k = k
            mx_w = bst_j
            
    # STOP IF INCREASE IS LESS THAN TOL
    inc = mx-old
    if inc<=TOL: 
        print(); print(f'{inc} is less. No increase. Stopping.')
        break
        
    # DISPLAY RESULTS
    print(); #print(kk,mx,mx_k,mx_w,'%.5f'%inc)
    print('Ensemble acc_score = %.4f after adding model %i with weight %.5f. Increase of %.5f'%(mx,mx_k,mx_w,inc))
    print()
    
    old = mx; m.append(mx_k); w.append(mx_w)

Ensemble acc_score = 0.9498 by beginning with model 0

(21397, 5)
Searching for best model to add... 
0 , 1 , 2 , 3 , 4 , 
Ensemble acc_score = 0.9523 after adding model 2 with weight 0.44500. Increase of 0.00252

(21397, 5)
Searching for best model to add... 
0 , 1 , 2 , 3 , 4 , 
4.6735523671626034e-05 is less. No increase. Stopping.


In [16]:
w

[0.445]

In [17]:
x.shape
model_avg = x.mean(axis=0)
print(model_avg.shape)
model_avg_predictions = np.argmax(model_avg,axis=1)
labels = TRUE
print(labels.shape)
print(f'Avg model prediction score = {accuracy_score(labels, model_avg_predictions)}')

(21397, 5)
(21397,)
Avg model prediction score = 0.936019068093658


In [18]:
np.save('oof_preds.npy', x)
np.save('oof_labels.npy', labels)

In [19]:
x[:,0,:]

array([[0.4287225 , 0.21637243, 0.19013453, 0.00523662, 0.1595339 ],
       [0.3503957 , 0.18372487, 0.18679062, 0.06341194, 0.21567686],
       [0.5320639 , 0.14809611, 0.15941694, 0.06226701, 0.09815604],
       [0.28049955, 0.14849293, 0.26587525, 0.02380693, 0.28132537],
       [0.34316963, 0.21031588, 0.15508245, 0.00797882, 0.2834532 ]])

In [20]:
output = np.zeros((x.shape[1], 25))
for i in range(x.shape[1]):
    output[i] = x[:,i,:].flatten()

In [21]:
output[0]

array([0.4287225 , 0.21637243, 0.19013453, 0.00523662, 0.1595339 ,
       0.3503957 , 0.18372487, 0.18679062, 0.06341194, 0.21567686,
       0.5320639 , 0.14809611, 0.15941694, 0.06226701, 0.09815604,
       0.28049955, 0.14849293, 0.26587525, 0.02380693, 0.28132537,
       0.34316963, 0.21031588, 0.15508245, 0.00797882, 0.2834532 ])